In [ ]:
def inspect_prediction(X, pred):
    pred_len = len(pred)
    n_signals = X.shape[1]
    
    figs, axs = plt.subplots(n_signals, 1, sharex=True, figsize=(30,30))
    for i, col in enumerate(TARGET_LABELS):
        axs[i].plot(np.arange(pred_len), X[-pred_len*2:-pred_len,i])
        axs[i].plot(np.arange(pred_len, 2*pred_len), X[-pred_len:,i], color='green')
        axs[i].plot(np.arange(pred_len, 2*pred_len), pred[:,i], color='orange')
        axs[i].set_title(col)
    plt.show()

In [ ]:
def get_batch_ending_at(X, end_index, in_len, batch_size, DEBUG = False):
    start_index = end_index - in_len * batch_size
    batch = []
    for i in range(start_index, end_index, in_len):
        batch.append(X[i : i + in_len])
        
        if DEBUG:
            print('[@get_batch_ending_at] window indexes:', i, i+in_len-1)
        
    return np.array(batch)
    

def autoreg_prediction(X,
                       model,
                       pred_len=864,
                       add_sin=True,
                       add_cos=True,
                       frequency = FREQUENCY,
                       batch_size = BATCH_SIZE,
                       DEBUG=False):
    
    X = np.array(X, dtype='float32')
    if DEBUG:
        f_name = 'autoreg_prediction'
        print(f'\n[@{f_name}] Converting tensor to np array')
        print('\tX starting shape:', X.shape)
        
    # Add sin and cosine columns
    if add_sin:
        sin_col = [np.sin(index * (2 * np.pi / frequency)) for index in range(len(X))]
        sin_col = np.expand_dims(sin_col, axis=-1)
        X = np.append(X, sin_col, axis=1)
        if DEBUG:
            print(f'\n[@{f_name}] Adding sin column to X')
            print('\tX shape after sin column addition:', X.shape)
    if add_cos:
        cos_col = [np.cos(index * (2 * np.pi / frequency)) for index in range(len(X))]
        cos_col = np.expand_dims(cos_col, axis=-1)
        X = np.append(X, cos_col, axis=1)
        if DEBUG:
            print(f'\n[@{f_name}] Adding cos column to X')
            print('\tX shape after cos column addition:', X.shape)
        
    
    model_in = model.layers[0].input_shape[-2]
    model_out = model.layers[-1].output_shape[-2]
    if DEBUG:
        print(f'\n[@{f_name}] Reading model input and output shapes')
        print('\tmodel_in:', model_in)
        print('\tmodel_out:', model_out)
    
    i = 0
    prediction = []
    while len(prediction) == 0 or len(prediction) < pred_len:
        i += 1
        
        batch = get_batch_ending_at(X, len(X), model_in, batch_size, DEBUG=False)
        if DEBUG:
            print(f'\n[@{f_name}] Predicting batch {str(i)}')
            print(f'\tshape batch {str(i)}:', batch.shape)

        temp_pred = model.predict(batch)[-1]
        if DEBUG:
            print(f'\tshape prediction {str(i)}:', temp_pred.shape)
        
        if add_sin:
            sin_col = [np.sin(index * (2 * np.pi / frequency)) for index in range(len(X), len(X) + len(temp_pred))]
            sin_col = np.expand_dims(sin_col, axis=-1)
            temp_pred = np.append(temp_pred, sin_col, axis=1)
            if DEBUG:
                print(f'\n[@{f_name}] Adding sin column to prediction {str(i)}')
                print('\tPrediction shape after sin column addition:', temp_pred.shape)
            
        if add_cos:
            cos_col = [np.cos(index * (2 * np.pi / frequency)) for index in range(len(X), len(X) + len(temp_pred))]
            cos_col = np.expand_dims(cos_col, axis=-1)
            temp_pred = np.append(temp_pred, cos_col, axis=1)
            if DEBUG:
                print(f'\n[@{f_name}] Adding cos column to prediction {str(i)}')
                print('\tPrediction shape after sin column addition:', temp_pred.shape)
        
        prediction = np.append(prediction, temp_pred, axis=0) if len(prediction) != 0 else temp_pred
        if DEBUG:
            print(f'\n[@{f_name}] Adding prediction {str(i)} to total prediction')
            print(f'\tPrediction shape after concatenating prediction {str(i)}:', prediction.shape)
        
        X = np.append(X, temp_pred, axis=0)
        if DEBUG:
            print(f'\n[@{f_name}] Adding prediction {str(i)} to dataframe')
            print('\tX shape after concatenating prediction:', X.shape)
            print('##################################################################')
            
    return prediction[0 : pred_len]

In [ ]:
model = tfk.models.load_model('***')
pred = autoreg_prediction(X, model, add_cos=False, add_sin=False)
tempX = X*std + mean
tempPred = pred*std + mean
inspect_prediction(tempX, tempPred)
error = np.sqrt((tempX[-864:] - tempPred)**2)
print('mse:',error.mean())

In [ ]:
# NB different from previous one
def get_batch_ending_at(X, end_index, in_len, batch_size, DEBUG=False):
    start_index = end_index - in_len * batch_size
    batch = []
    for i in range(start_index-1, end_index-1, in_len):
        batch.append(X[i : i + in_len])
        
        if DEBUG:
            print('[@get_batch_ending_at] window indexes:', i, i+in_len-1)
        
    return np.array(batch)

def get_median_value(pred_vals, index, model_out, DEBUG=False):
    if DEBUG:
        print('[@get_median_value] index:', index)
        print(f'\tPredictions {str(index)} shape: {np.array(pred_vals[index]).shape}')
    
    if not index in pred_vals.keys():
        print(f'index {str(index)} not valid')
        assert index in pred_vals.keys()
    
    if not len(pred_vals[index]) == model_out:
        print(f'index {str(index)}: wrong number of samples: {str(len(pred_vals))}, should be: {str(model_out)}')
        len(pred_vals[index]) == model_out
    pred = np.array(pred_vals[index])
    pred = np.median(pred, axis=0)
    return pred

def overlapping_autoreg_prediction(X,
                                   model,
                                   pred_len=1152,
                                   add_sin=True,
                                   add_cos=True,
                                   frequency = FREQUENCY,
                                   batch_size = BATCH_SIZE,
                                   DEBUG=False):
    
    f_name = 'overlapping_autoreg_prediction'
    
    X = np.array(X, dtype='float32')
    if DEBUG:
        print(f'\n[@{f_name}] Converting tensor to np array')
        print('\tX starting shape:', X.shape)
        
    # Add sin and cosine columns
    if add_sin:
        sin_col = [np.sin(index * (2 * np.pi / frequency)) for index in range(len(X))]
        sin_col = np.expand_dims(sin_col, axis=-1)
        X = np.append(X, sin_col, axis=1)
        if DEBUG:
            print(f'\n[@{f_name}] Adding sin column to X')
            print('\tX shape after sin column addition:', X.shape)
    if add_cos:
        cos_col = [np.cos(index * (2 * np.pi / frequency)) for index in range(len(X))]
        cos_col = np.expand_dims(cos_col, axis=-1)
        X = np.append(X, cos_col, axis=1)
        if DEBUG:
            print(f'\n[@{f_name}] Adding cos column to X')
            print('\tX shape after cos column addition:', X.shape)
        
    
    model_in = model.layers[0].input_shape[-2]
    model_out = model.layers[-1].output_shape[-2]
    if DEBUG:
        print(f'\n[@{f_name}] Reading model input and output shapes')
        print('\tmodel_in:', model_in)
        print('\tmodel_out:', model_out)
        
    pred_start_index = len(X)
    pred_end_index = pred_start_index + pred_len
    pred_vals = {i:[] for i in range(pred_start_index, pred_end_index)}
    if DEBUG:
        print(f'\n[@{f_name}] Calculating prediction indexes')
        print('\tpred_start_index:', pred_start_index)
        print('\tpred_end_index:', pred_end_index)
        
    # First part (No Dataset Modifications)
    for i, batch_end in enumerate(range(len(X) - (model_out-1), len(X)+1)):
        
        batch = get_batch_ending_at(X, batch_end, model_in, batch_size, DEBUG=False)
        if DEBUG:
            print(f'\n[@{f_name}] Predicting batch {str(i)}')
            print(f'\tshape batch {str(i)}:', batch.shape)
            
        temp_pred = model.predict(batch)[-1]
        if DEBUG:
            print(f'\tshape prediction {str(i)}:', temp_pred.shape)
            print('---------------------------------------------------------------')
            
        for j in range(model_out):
            if (batch_end + j) in pred_vals.keys():
                pred_vals[batch_end + j].append(temp_pred[j])
    
    # Second part (lenghten dataset with predictions)
    new_val_index = pred_start_index
    for batch_end in range(pred_start_index, pred_end_index+1):
        if DEBUG:
            print(f'\n[@{f_name}] Concatenating item {str(new_val_index)} to dataset')
            
        if new_val_index != pred_end_index:
            new_val = get_median_value(pred_vals, new_val_index, model_out, DEBUG=False)
        
            if add_sin:
                new_val = np.append(new_val, np.sin(new_val_index * (2 * np.pi / frequency)))
            if add_cos:
                new_val = np.append(new_val, np.cos(new_val_index * (2 * np.pi / frequency)))

            new_val = np.expand_dims(new_val, axis=0)
            if DEBUG:
                print('\tItem shape:', new_val.shape)

            X = np.append(X, new_val, axis=0)
            if DEBUG:
                print('\tX shape:', X.shape)
        
        batch = get_batch_ending_at(X, batch_end, model_in, batch_size, DEBUG=False)
        if DEBUG:
            print(f'\n[@{f_name}] Predicting batch ending at {str(batch_end)}')
            
        temp_pred = model.predict(batch)[-1]
        if DEBUG:
            print('\tPrediction shape:', temp_pred.shape)
            
        for j in range(model_out):
            if (batch_end + 1 + j) in pred_vals.keys():
                pred_vals[batch_end + 1 + j].append(temp_pred[j])
        
        new_val_index += 1
        
    
    
    # Compute prediction as median
    final_pred = [[] for i in range(pred_len)]
    for i in pred_vals.keys():
        pred_vals[i] = np.array(pred_vals[i])
        final_pred[i - pred_start_index] =  get_median_value(pred_vals, i, model_out, DEBUG=False)
    
    final_pred = np.array(final_pred)
    if DEBUG:
        print(f'\n[@{f_name}] Final Prediction shape: {final_pred.shape}')
    return final_pred

In [ ]:
model = tfk.models.load_model('***')
pred = overlapping_autoreg_prediction(X[:-864], model, add_cos=False, add_sin=False)
inspect_prediction(X*std+mean, pred*std+mean)
tempX = X*std + mean
tempPred = pred*std + mean
error = np.sqrt((tempX[-864:] - tempPred)**2)
print('mse:',error.mean())